In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

In [55]:
#Hyperparameters
image_size = 356
total_steps = 6000
learning_rate = 0.001
alpha = 0.98
beta = 0.02

In [44]:
loader = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available else "gpu")
model = models.vgg19(pretrained = True)
model

In [46]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()

        self.chosen_features = ["0", "5", "10", "19", "28"]
        self.model = models.vgg19(pretrained = True).features[:29]

    def forward(self, x):
        features = []
        for layer_num, layer in enumerate(self.model):
            x = layer(x)

            if str(layer_num) in self.chosen_features:
                features.append(x)
        return features

In [60]:
model = VGG().to(device).eval()
def load_image(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device)

In [61]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()

        self.chosen_features = ["0", "5", "10", "19", "28"]
        self.model = models.vgg19(pretrained = True).features[:29]

    def forward(self, x):
        features = []
        for layer_num, layer in enumerate(self.model):
            x = layer(x)

            if str(layer_num) in self.chosen_features:
                features.append(x)
        return features

In [62]:
original_img = load_image("/content/IMAGES/CRY.jpg")

In [63]:
generated = original_img.clone().requires_grad_(True)

In [64]:
optimizer = optim.Adam([generated], lr = learning_rate)

In [ ]:
dots = "-"*50
while True:
    type_img = int(input(f"Which painter's painting style would you like to transform your painting into (Accept only numbers without '-') \n {dots} \n 1- Van Gogh \n 2- Leonardo Da Vinci \n 3- Pablo Picasso \n"))
    if type_img == 1:
        !wget -O VANGOGH.jpg https://th.bing.com/th/id/R.484fcbc7078d68106d9e69bebfba553d?rik=KxeNXAWCRjEkbw&pid=ImgRaw&r=0
        style_img = load_image("/content/IMAGES/VANGOGH.jpeg")
        break

    elif type_img == 2:
        !wget -O LEONARDO.jpg https://1.bp.blogspot.com/-VVSlaRqBfFY/XnimN6ksdRI/AAAAAAAAF_Q/GSB3boip000o6OiX5mNgNQZBqEdWVkAygCLcBGAsYHQ/s1600/16-54-37-Da_Vinci%2527s_Mona_Lisa_with_original_colors_approximation.jpg
        style_img = load_image("/content/IMAGES/LEONARDO.jpeg")
        break

    elif type_img == 3:
        !wget -O PICASSO.jpg https://th.bing.com/th/id/OIP.gzMFhLT6IWde2sN8dUcH9AHaHe?pid=ImgDet&rs=1
        style_img = load_image("/content/IMAGES/PICASSO.jpeg")
        break

    else:
        print(f"Try again later with available NUMBERS \n {dots} \n")
        continue

In [66]:
for step in range(total_steps):
    generated_features = model(generated)
    original_img_features = model(original_img)
    style_features = model(style_img)

    style_loss = original_loss = 0

    for gen_feature, orig_feature, style_feature in zip(
        generated_features, original_img_features, style_features
        ):

        batch_size, channel, height, width = gen_feature.shape
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)

        #Compute Gram Matrix

        G = gen_feature.view(channel, height*width).mm(
            gen_feature.view(channel, height*width).t()
        )

        A = style_feature.view(channel, height*width).mm(
            style_feature.view(channel, height*width).t()
        )

        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step % 200 == 0:
        print(total_loss)
        save_image(generated, "generated.png")

tensor(1483128., device='cuda:0', grad_fn=<AddBackward0>)
tensor(145120.7188, device='cuda:0', grad_fn=<AddBackward0>)
tensor(77870.3281, device='cuda:0', grad_fn=<AddBackward0>)
tensor(46068.2031, device='cuda:0', grad_fn=<AddBackward0>)
tensor(29235.4023, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20773.2227, device='cuda:0', grad_fn=<AddBackward0>)
tensor(16545.8965, device='cuda:0', grad_fn=<AddBackward0>)
tensor(14193.5869, device='cuda:0', grad_fn=<AddBackward0>)
tensor(12639.7705, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11483.7900, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10557.2920, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9791.4375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9138.9941, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8574.1133, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: ignored